In [1]:
# import packages 
import requests
import json
from IPython.display import JSON


In [4]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"

In [5]:
# send the request to the API
res_air = requests.get(url)
JSON(res_air.json())

<IPython.core.display.JSON object>

In [6]:
# check if the request was successful
res_air.status_code

200

In [8]:
# export the information that was returned using method .json()
info=res_air.json()
info.keys()
for key, value in info.items():

    print(key)

$id
$type
updatePeriod
updateFrequency
forecastURL
disclaimerText
currentForecast


## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [9]:
# weather forecast tommorrow text 
weather_tommorrow=requests.get('https://api.tfl.gov.uk/AirQuality?dates=2010-09-18')
info1=weather_tommorrow.json()
print("tommorrow weather forcast: ",info1['currentForecast'][1]['forecastText'])

tommorrow weather forcast:  A continuing feed of air from continental Europe into Greater London is forecast by back-trajectory models for Saturday. As on Friday though, a marked import of long-range particulates is not currently expected as the predicted speed and altitude of the incoming air will limit the potential for it to accumulate pollutants.&lt;br/&gt;Again, a good easterly breeze should give good dispersion of local emissions.&lt;br/&gt;Similar forecast is for Sunday. &lt;br/&gt;Dry and sunny on Monday with lighter winds from the east. Pollution levels for Monday will be reviewed over the weekend  &lt;br/&gt;&lt;br/&gt;Air pollution is expected to remain &#39;Low&#39; throughout the forecast period for the following pollutants:&lt;br/&gt;&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;Sulphur Dioxide&lt;br/&gt;Ozone&lt;br/&gt;PM2.5 Particulates&lt;br/&gt;PM10 Particulates&lt;br/&gt;&lt;br/&gt;


In [10]:
#find the total numbers of tranport modes"
from IPython.display import JSON
url1=('https://api.tfl.gov.uk/Line/Meta/Modes')
transport=requests.get(url1)
transport_json=transport.json()
transport_json
transport_mode=[]

for item in transport_json:
     for key, value in item.items():
          if key=='modeName':
              transport_mode.append(value)

uni_transport=set(transport_mode)
print(f'Number of different modes of transport is: {len(uni_transport)}')
        
    


Number of different modes of transport is: 18


## How many tube and bus lines are in London? Print names of all tube lines.


In [37]:
#number of tubes
tube_url=('https://api.tfl.gov.uk/Line/Mode/tube')
tube_response=requests.get(tube_url)
tube_json=tube_response.json()
tube_name=[]
for item in tube_json:
    tube_name.append(item['name'])
print(f'The london tube names are {tube_name}')


The london tube names are ['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


In [12]:
# Total Number of Bus Names 
bus_url=('https://api.tfl.gov.uk/Line/Mode/bus')
bus_response=requests.get(bus_url)
bus_json=bus_response.json()
bus_name=[]
for item in bus_json:
    bus_name.append(item['name'])
print(f'The total london bus routs are {len(bus_name)}')

The total london bus routs are 681


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [25]:
#numbers of bikepoints locations =total ID

url2=('https://api.tfl.gov.uk/BikePoint')
bikepoints=requests.get(url2)
#bikepoints.headers
#bikepoints.content
bikepointslist=[]
bikepoints_json=bikepoints.json()
for item in bikepoints_json:
    bikepointsID=item['id']
    bikepointslist.append(bikepointsID)
print(len(bikepointslist))

784


In [40]:
#bikepoints_json[1]['additionalProperties']

In [32]:
# number of docks of bikepoints (#val['key']=='NbEmptyDocks'(empty docks will not be included)
docks=[]
sum_docks=0

for item in bikepoints_json:
    for val in item['additionalProperties']:
        if val['key']=='NbDocks': 
            sum_docks+=int(val['value'])                                          
print(sum_docks)

20853


## Task
How many station has `victoria` line?

In [19]:
url4=('https://api.tfl.gov.uk/Line/victoria/StopPoints')

In [25]:
stops=requests.get(url4)
stops_json=stops.json()
stops_list=[]
for item in stops_json:
    for key, value in item.items():
        if key=='stationNaptan':
            stops_list.append(value)
print(len(stops_list))



16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Public Transport, Taxi or Bike? Which way is the fastest? Example output:
```
Planned duration:
Public Transport: x minutes
Bike: y minutes
```

We need to search the documentation for correct requests.

In [17]:
url5=('https://api.tfl.gov.uk/journey/journeyresults/51.5039580037,-0.48810008749/to/51.508139082110006, -0.54784242932')

In [18]:
# search bus time
bus_response=requests.get(url5,params={'mode':'bus'})
bus_json=bus_response.json()
bus_json.keys()


dict_keys(['$type', 'journeys', 'lines', 'stopMessages', 'recommendedMaxAgeMinutes', 'searchCriteria', 'journeyVector'])

In [19]:
# extract bus time
import datetime
bus_info=bus_json['journeys'][0]
#print(bus_info)
duration=bus_info['duration']
arrive_time=datetime.datetime.strptime((bus_info['arrivalDateTime']),'%Y-%m-%dT%H:%M:%S')
start_time=datetime.datetime.strptime((bus_info['startDateTime']),'%Y-%m-%dT%H:%M:%S')
minutes=arrive_time-start_time
#print(minutes)


from pprint import pprint  
pprint(f"the bus duration is {duration}, and the recommended time is {minutes}")


'the bus duration is 85, and the recommended time is 1:26:00'


In [23]:
# extract bike time
bike=requests.get(url5,params={'mode':'cycle'})
bike_json=bike.json()
bike_info=bike_json['journeys'][0]

#pprint(bike_json)

duration=bike_info['duration']
arrive_time=datetime.datetime.strptime((bike_info['arrivalDateTime']),'%Y-%m-%dT%H:%M:%S')
start_time=datetime.datetime.strptime((bike_info['startDateTime']),'%Y-%m-%dT%H:%M:%S')
minutes=arrive_time-start_time
pprint(f"the bike duration is {duration}, and the recommended time is {minutes}")

'the bike duration is 18, and the recommended time is 0:18:00'


In [96]:
# extract taxi time
taxi=requests.get(url5,params={'mode':'taxi'})
taxi_json=taxi.json()


In [99]:
taxi_info=taxi_json['journeys'][0]
#print(taxi_info)
duration1=taxi_info['duration']
arrive_time=datetime.datetime.strptime((taxi_info['arrivalDateTime']),'%Y-%m-%dT%H:%M:%S')
start_time=datetime.datetime.strptime((taxi_info['startDateTime']),'%Y-%m-%dT%H:%M:%S')
taxi_time=arrive_time-start_time
  
print(f"the taxi duration is {duration1}, and the required time is {taxi_time}")

the taxi duration is 75, and the required time is 1:15:00
